In [1]:
from datetime import datetime
import pandas as pd
import requests
import numpy as np
from forex_python.converter import get_rate

In [ ]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\part-00000-67beda92-080e-4801-bee9-784943534377-c000.csv')

In [ ]:
timestamps1 = [int(datetime.strptime(time, '%Y-%m-%d %H:%M:%S').timestamp()*1000) for time in data.UTC_Time]
timestamps2 = [time + 59999 for time in timestamps1]

In [ ]:
i = 10
tick_interval = '1m'

prices = []
markets = []
markets1 = []
prices1 = []
for i in range(len(timestamps1)):
    price1 = '1'
    market1 = 'None'
    print(i, end='\r')
    if data.Coin[i] not in ['EUR', 'NFT']:
        try:
            market = data.Coin[i]+'EUR'
            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
            price = requests.get(url).json()[0][1]
            
        except:
            try:
                market = 'EUR'+data.Coin[i]
                url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                price = str(1/float(requests.get(url).json()[0][1]))
                market = data.Coin[i]+'EUR'
                
            except:
                try:
                    market = data.Coin[i]+'USDT'
                    market1 = 'EURUSDT'
                    url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                    price = requests.get(url).json()[0][1]
                    
                    url1 = 'https://api.binance.com/api/v3/klines?symbol=EURUSDT&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                    price1 = requests.get(url1).json()[0][1]
                
                except:
                    try:
                        market = data.Coin[i]+'BUSD'
                        market1 = 'EURBUSD'
                        url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                        price = requests.get(url).json()[0][1]

                        url1 = 'https://api.binance.com/api/v3/klines?symbol=EURBUSD&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                        price1 = requests.get(url1).json()[0][1]
                        
                    except:
                        try:
                            market = data.Coin[i] + 'BTC'
                            market1 = 'EURBTC'
                            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                            price = requests.get(url).json()[0][1]

                            url1 = 'https://api.binance.com/api/v3/klines?symbol=BTCEUR&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                            price1 = str(1/float(requests.get(url1).json()[0][1]))
                            
                        except:
                            market = data.Coin[i]+'USDT'
                            market1 = 'EURUSDT'
                            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])
                            price = requests.get(url).json()[0][1]

                            url1 = 'https://api.binance.com/api/v3/klines?symbol=EURUSDT&interval='+tick_interval+'&startTime='+str(requests.get(url).json()[0][0])
                            price1 = requests.get(url1).json()[0][1]

                            print(requests.get(url).json()[0][0])
                            print(requests.get(url1).json()[0][0])
                            print(data.Coin[i])
                            
    elif data.Coin[i] == 'EUR':
        price = '1'
        market = 'EUR'
    elif data.Coin[i] == 'NFT':
        price = '0'
        market = 'NFT'
    prices.append(price)
    prices1.append(price1)
    markets.append(market)
    markets1.append(market1)

In [ ]:
UnitPrices = [float(prices[i]) / float(prices1[i]) for i in range(len(prices))]
totalPrices = [UnitPrices[i] * float(data.Change[i]) for i in range(len(prices)) ]

In [ ]:
data1 = pd.DataFrame({'Unit Prices': UnitPrices, 'Total Price (EUR)': totalPrices})
#'Tool':markets, 'Tool Price':prices, 'Tool to EUR': markets1, 
#                       'Coefficient Price': prices1, 

In [ ]:
Data = pd.concat([data, data1], axis=1)

In [ ]:
Data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_EUR.csv')

In [2]:
Data=pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_EUR.csv')

In [3]:
uniqueCoins = np.unique(Data.Coin)
uniqueCoins

array(['ADA', 'ALICE', 'BAR', 'BAT', 'BNB', 'BUSD', 'CRV', 'DOGE', 'ETH',
       'EUR', 'IOTA', 'LUNA', 'MANA', 'QI', 'SFP', 'SHIB', 'USDT', 'WABI'],
      dtype=object)

Calculate the spot by the end of the year

In [5]:
for i in uniqueCoins:
    print(i +':   ', np.round(np.sum(Data.Change[Data.Coin==i]),10))

ADA:    95.15150236
ALICE:    0.0
BAR:    0.0
BAT:    0.0
BNB:    1.58643072
BUSD:    0.0
CRV:    17.5
DOGE:    0.2767986
ETH:    0.0
EUR:    -0.0
IOTA:    203.833
LUNA:    0.0
MANA:    70.0
QI:    7.36098866
SFP:    0.0
SHIB:    5185519.199999999
USDT:    2.11259859
WABI:    0.0


In [6]:
for i in uniqueCoins:
    print(i +':   ', np.round(np.sum(Data['Total Price (EUR)'][Data.Coin==i]),10))

ADA:    143.2549866151
ALICE:    0.0052673485
BAR:    0.0
BAT:    -51.6147632859
BNB:    220.7878373527
BUSD:    -0.1187395808
CRV:    92.6828724353
DOGE:    49.3721849666
ETH:    -106.2022101428
EUR:    -0.0
IOTA:    165.3203980797
LUNA:    -6.3964919372
MANA:    180.6271289556
QI:    1.1015426393
SFP:    -27.7216191139
SHIB:    224.5338609278
USDT:    -1.854270411
WABI:    111.1158575715


In [8]:
Data["UTC_Time"] = pd.to_datetime(Data["UTC_Time"], format='%Y-%m-%d %H:%M:%S')
Data = Data.sort_values(by="UTC_Time")

Data

,Unnamed: 0,User_ID,UTC_Time,Account,Operation,Coin,Change,Remark,Unit Prices,Total Price (EUR)
0,0,71190841,2021-02-08 08:33:27,Spot,Deposit,EUR,100.000000,NaN,1.000000,100.000000
1,1,71190841,2021-02-08 08:44:50,Spot,Buy,DOGE,418.000000,NaN,0.058370,24.398660
2,2,71190841,2021-02-08 08:44:50,Spot,Transaction Related,EUR,-24.871000,NaN,1.000000,-24.871000
3,3,71190841,2021-02-08 08:44:53,Spot,Transaction Related,EUR,-0.047600,NaN,1.000000,-0.047600
4,4,71190841,2021-02-08 08:44:53,Spot,Buy,DOGE,0.800000,NaN,0.058370,0.046696
...,...,...,...,...,...,...,...,...,...,...
726,726,71190841,2021-12-14 14:53:50,Spot,Fee,BNB,-0.000075,NaN,459.300000,-0.034544
729,729,71190841,2021-12-16 00:16:24,Spot,Launchpool Interest,QI,7.360989,NaN,0.149646,1.101543
731,731,71190841,2021-12-19 01:08:23,Spot,Buy,USDT,-49.938000,NaN,0.890472,-44.468388
730,730,71190841,2021-12-19 01:08:23,Spot,Buy,MANA,14.000000,NaN,3.108994,43.525913


In [9]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': Data['Change'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'date_buy': Data['UTC_Time'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'price_buy': Data['Total Price (EUR)'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'amount_sell': Data['Change'][(Data.Coin==coin) & (Data.Change<0)].to_list(),
                'date_sell': Data['UTC_Time'][(Data.Coin==coin) & (Data.Change<0)].to_list(),
                'price_sell': Data['Total Price (EUR)'][(Data.Coin==coin) & (Data.Change<0)].to_list()}

In [10]:
ALL

{'ADA': {'amount_buy': [27.5,
   67.52,
   49.51,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   57.4,
   42.6,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   104.6,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   117.03,
   127.7,
 

In [11]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [12]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [13]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%Y-%m-%d %H:%M:%S')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%Y-%m-%d %H:%M:%S')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)
Report = Report[Report['Sell Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

In [14]:
Report.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Report.csv')

In [15]:
Deposits = Data[Data.Operation == 'Deposit'].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Deposits['Total Price (EUR)'] = np.round(Deposits['Total Price (EUR)'], 2)
Deposits['Fee (EUR)'] = [0.0]*len(Deposits['Total Price (EUR)'])
# Deposits.iloc[[8, 9, 13, 14], 6] = [0.2, 4.4, 5.57, 13.26]
Deposits.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)', 'Fee (EUR)']
Deposits["Date"] = pd.to_datetime(Deposits["Date"], format='%Y-%m-%d %H:%M:%S')
Deposits = Deposits.sort_values(by="Date").reset_index(drop=True)
Deposits = Deposits[Deposits['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)
Deposits

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR),Fee (EUR)
0,2021-02-08 08:33:27,Deposit,EUR,100.0,1.0,100.0,0.0
1,2021-02-16 09:45:09,Deposit,EUR,100.0,1.0,100.0,0.0
2,2021-05-13 23:27:09,Deposit,EUR,98.2,1.0,98.2,0.0
3,2021-05-19 07:20:29,Deposit,EUR,196.4,1.0,196.4,0.0
4,2021-05-19 16:10:23,Deposit,EUR,196.4,1.0,196.4,0.0
5,2021-05-23 11:35:19,Deposit,EUR,196.4,1.0,196.4,0.0
6,2021-06-22 15:28:54,Deposit,EUR,196.4,1.0,196.4,0.0


In [16]:
Deposits.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Deposits.csv')

In [17]:
Withdraw = Data[Data.Operation.isin(['Withdraw', 'Binance Card Spending'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Unnamed: 0'],axis=1)
Withdraw = Withdraw.fillna('No Fee')
Withdraw['Total Price (EUR)'] = -np.round(Withdraw['Total Price (EUR)'], 2)
Withdraw['Change'] *= -1
Withdraw.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Fee (EUR)', 'Unit Price (EUR)', 'Total Price (EUR)']
Withdraw = Withdraw [['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)', 'Fee (EUR)']]
Withdraw["Date"] = pd.to_datetime(Withdraw["Date"], format='%Y-%m-%d %H:%M:%S')
Withdraw = Withdraw.sort_values(by="Date").reset_index(drop=True)
Withdraw = Withdraw[Withdraw['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

Withdraw

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR),Fee (EUR)


In [19]:
Withdraw.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Withdrawals.csv')

In [20]:
Fee = Data[Data.Operation.isin(['Fee'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Fee['Total Price (EUR)'] = -np.round(Fee['Total Price (EUR)'], 2)
Fee['Change'] *= -1
Fee.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)']
Fee["Date"] = pd.to_datetime(Fee["Date"], format='%Y-%m-%d %H:%M:%S')
Fee = Fee.sort_values(by="Date").reset_index(drop=True)
Fee = Fee[Fee['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)


Fee

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR)
0,2021-02-08 14:36:19,Fee,SFP,0.021230,1.060316,0.02
1,2021-02-08 16:41:00,Fee,LUNA,0.004010,4.058095,0.02
2,2021-02-08 21:57:50,Fee,ADA,0.027500,0.589000,0.02
3,2021-02-18 07:59:28,Fee,BNB,0.000133,147.700000,0.02
4,2021-02-18 08:02:12,Fee,BNB,0.000131,147.285800,0.02
...,...,...,...,...,...,...
154,2021-11-26 14:01:54,Fee,BNB,0.000285,523.700000,0.15
155,2021-12-14 11:48:25,Fee,BNB,0.000051,462.400000,0.02
156,2021-12-14 11:48:25,Fee,BNB,0.000031,462.400000,0.01
157,2021-12-14 14:53:50,Fee,BNB,0.000075,459.300000,0.03


In [21]:
Fee.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Fees.csv')

In [22]:
Cashback = Data[Data.Operation.isin(['Card Cashback', 'Commission Rebate', 'Launchpool Interest'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Cashback['Total Price (EUR)'] = np.round(Cashback['Total Price (EUR)'], 2)
Cashback['Change'] *= -1
Cashback.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)']
Cashback["Date"] = pd.to_datetime(Cashback["Date"], format='%Y-%m-%d %H:%M:%S')
Cashback = Cashback.sort_values(by="Date").reset_index(drop=True)
Cashback = Cashback[Cashback['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

Cashback

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR)
0,2021-04-09 00:30:07,Launchpool Interest,ALICE,-0.177064,10.727387,1.9
1,2021-12-16 00:16:24,Launchpool Interest,QI,-7.360989,0.149646,1.1


In [23]:
Cashback.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Cashbacks.csv')

In [ ]:
np.unique(Data.Operation)

In [24]:
sum(profit) + sum(Cashback['Total Price (EUR)']) -sum(loss)-sum(Fee['Total Price (EUR)']) - sum(Deposits['Fee (EUR)'])

840.3699999999974

In [25]:
sum(Withdraw['Total Price (EUR)'])-sum(Deposits['Total Price (EUR)'])

-1083.8

In [26]:
sum(Deposits['Total Price (EUR)'])

1083.8

In [27]:
sum(loss)

567.0999999999998

In [28]:
sum(Fee['Total Price (EUR)']) + sum(Deposits['Fee (EUR)'])

15.419999999999986

In [29]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

ADA :    95.15150
ALICE :    0.00000
BAR :    0.00000
BAT :    0.00000
BNB :    1.58643
BUSD :    0.00000
CRV :    17.50000
DOGE :    0.27680
ETH :    0.00000
EUR :    0.00000
IOTA :    203.83300
LUNA :    0.00000
MANA :    70.00000
QI :    7.36099
SFP :    0.00000
SHIB :    5185519.20000
USDT :    2.11260
WABI :    0.00000


# Etoro

In [36]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro.csv')

In [37]:
data

,Position ID,Action,Amount,Units,Open Date,Close Date,Leverage,Spread,Profit,Open Rate,Close Rate,Take profit rate,Stop lose rate,Rollover Fees and Dividends,Copied From,Type,ISIN,Notes
0,793962426,Buy Xiaomi Corp,10.01,3.331238,13/11/2020 01:30:11,30/12/2020 01:30:20,1,0.00,4.06,23.30,32.75,248.56,0.01,0.0,LG2001,Stocks,KYG9830T1067,NaN
1,816766394,Buy Xiaomi Corp,3.52,1.123064,02/12/2020 07:04:09,30/12/2020 01:30:20,1,0.00,1.22,24.30,32.75,267.28,0.01,0.0,LG2001,Stocks,KYG9830T1067,NaN
2,847419499,Buy Nano Dimension Ltd,1.29,0.123682,23/12/2020 17:18:37,29/12/2020 14:35:45,1,0.00,-0.27,10.43,8.24,0.00,0.01,0.0,LG2001,Stocks,US63008G2030,NaN
3,793480168,Buy Alibaba,6.01,0.022577,12/11/2020 16:52:39,29/12/2020 14:35:20,1,0.00,-0.82,266.20,230.09,"2,928.42",0.00,0.0,LG2001,Stocks,US01609W1027,NaN
4,793480163,Buy Alibaba,2.00,0.007513,12/11/2020 16:52:39,29/12/2020 14:35:20,1,0.00,-0.27,266.20,230.09,"2,937.44",0.00,0.0,LG2001,Stocks,US01609W1027,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,800952174,Buy Aston Martin Lagonda Global Holdings PLC,120.98,6.247181,20/11/2020 09:43:09,12/01/2021 14:34:02,1,0.00,19.83,"1,446.00","1,679.40",0.00,0.20,0.0,-,Stocks,GB00BN7CG237,NaN
650,801015160,Buy Aston Martin Lagonda Global Holdings PLC,120.00,6.305629,20/11/2020 13:16:43,12/01/2021 14:34:00,1,0.00,21.05,"1,434.00","1,679.40",0.00,0.20,0.0,-,Stocks,GB00BN7CG237,NaN
651,892979086,Buy Ethereum,110.00,0.099325,12/01/2021 11:51:47,12/01/2021 14:16:56,1,1.99,-6.14,"1,107.47","1,045.70",0.00,0.00,0.0,-,Crypto,NaN,NaN
652,892979220,Buy XRP,110.00,369.623656,12/01/2021 11:52:17,12/01/2021 14:16:47,1,2.55,-5.03,0.30,0.28,0.00,0.00,0.0,-,Crypto,NaN,NaN


In [39]:
USDEUR = []
for i, time in enumerate(data['Open Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i, end='\r')

In [40]:
data['Amount (EUR)'] = np.round(USDEUR * data['Amount'],2)

In [41]:
USDEUR = []
for i, time in enumerate(data['Close Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i, end='\r')

In [42]:
data['Amount Sell (EUR)'] = np.round((data['Amount'] + data['Profit']) * USDEUR,2)


data['Profit (EUR)'] = data['Amount Sell (EUR)'] - data['Amount (EUR)']

data['Action'] = [stock.replace('Buy ','') for stock in data['Action']]

In [43]:
data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro EUR.csv')

In [44]:
data_buy = data[['Action', 'Units', 'Open Date', 'Type', 'Amount (EUR)', 'Amount']]
data_buy.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)', 'Total Price (USD)']
data_buy['Action'] = ['Buy']*len(data_buy.Instrument)


data_sell = data[['Action', 'Units', 'Close Date', 'Type', 'Amount Sell (EUR)']]
data_sell.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)']
data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
data_sell['Action'] = ['Sell']*len(data_sell.Instrument)

data = pd.concat([data_buy, data_sell], axis=0)

data = data[['Type', 'Action', 'Instrument', 'Units', 'Date', 'Total Price (EUR)', 'Total Price (USD)']]

C:\Users\JKHATRI\AppData\Local\Temp\ipykernel_15820\18286657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_buy['Action'] = ['Buy']*len(data_buy.Instrument)
C:\Users\JKHATRI\AppData\Local\Temp\ipykernel_15820\18286657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
C:\Users\JKHATRI\AppData\Local\Temp\ipykernel_15820\18286657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [45]:
data["Date"] = pd.to_datetime(data["Date"], format='%d/%m/%Y %H:%M:%S')
data = data.sort_values(by="Date")

data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Buy Sell EUR.csv')

In [12]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Buy Sell EUR.csv')
data["Date"] = pd.to_datetime(data["Date"], format='%Y-%m-%d %H:%M:%S')

In [13]:
uniqueInstruments = np.unique(data.Instrument)
uniqueInstruments

array(['20+ Year Trsy Bond Ishares', '9F Inc', 'AIRBUS SE',
       'AMC Entertainment Holdings Inc', 'AYRO Inc',
       'Advanced Micro Devices Inc', 'Air France-KLM', 'Airbnb Inc',
       'Alibaba', 'Amazon.com Inc', 'Aphria Inc.', 'Apple', 'Asana',
       'Asensus Surgical Inc.',
       'Aston Martin Lagonda Global Holdings PLC', 'Athenex Inc',
       'Aurora Cannabis Inc', 'Ballard Power Systems Inc.',
       'Barrick Gold', 'Basic Attention Token', 'Bed Bath & Beyond Inc',
       'BioNTech SE', 'BioNano Genomics Inc', 'Bitcoin',
       'Blue Apron Holdings Inc.', 'Boeing', 'Boxlight Corp',
       'Canadian Solar Inc.', 'Cardano', 'Cenntro Electric Group Limited',
       'Cineworld Group PLC', 'Cloudflare', 'Cocrystal Pharma Inc',
       'Coinbase Global Inc', 'Consumer Discret Select Sector SPDR',
       'Corsair Gaming Inc', 'Crowdstrike Holdings', 'Curve',
       'Decentraland', 'Discovery Inc', 'Dogecoin', 'Dutch Bros Inc.',
       'ElringKlinger', 'Enjin', 'Ethereum', 'Evofem B

In [14]:
ALL ={}

for instrument in uniqueInstruments:
    ALL[instrument] ={'amount_buy': data['Units'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'date_buy': data['Date'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'price_buy': data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'amount_sell': (-data['Units'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list(),
                      'date_sell': data['Date'][(data.Instrument==instrument) & (data.Action == 'Sell')].to_list(),
                      'price_sell': (-data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list()}

In [15]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
stock = []
profit = []
loss = []
result = []
for instrument in uniqueInstruments:
    while (len(All_copy[instrument]['amount_buy']) != 0) and (len(All_copy[instrument]['amount_sell']) != 0):
        buy_date.append(All_copy[instrument]['date_buy'][0])
        sell_date.append(All_copy[instrument]['date_sell'][0])
        stock.append(instrument)
        if All_copy[instrument]['amount_buy'][0] <= abs(All_copy[instrument]['amount_sell'][0]):
            
            amount.append(All_copy[instrument]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[instrument]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[instrument]['price_sell'][0] * 
                                All_copy[instrument]['amount_buy'][0] / All_copy[instrument]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[instrument]['amount_sell'][0] += All_copy[instrument]['amount_buy'][0]
            All_copy[instrument]['price_sell'][0] += sellPricePortion
            
            if All_copy[instrument]['amount_sell'][0] == 0:
                del All_copy[instrument]['amount_sell'][0]
                del All_copy[instrument]['price_sell'][0]
                del All_copy[instrument]['date_sell'][0]
            del All_copy[instrument]['amount_buy'][0]
            del All_copy[instrument]['price_buy'][0]
            del All_copy[instrument]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[instrument]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[instrument]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[instrument]['price_buy'][0] * 
                                All_copy[instrument]['amount_sell'][0] / All_copy[instrument]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[instrument]['amount_buy'][0] += All_copy[instrument]['amount_sell'][0]
            All_copy[instrument]['price_buy'][0] -= buyPricePortion
            
            if All_copy[instrument]['amount_buy'][0] == 0:
                del All_copy[instrument]['amount_buy'][0]
                del All_copy[instrument]['price_buy'][0]
                del All_copy[instrument]['date_buy'][0]
            del All_copy[instrument]['amount_sell'][0]
            del All_copy[instrument]['price_sell'][0]
            del All_copy[instrument]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = abs(result *(result>=0))
loss = abs(result * (result<0))

In [16]:
Report =pd.DataFrame({'Stock': stock,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [17]:
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [18]:
Report.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Report.csv')

In [19]:
Stocks = np.unique(data.Instrument[data.Type=='Stocks'])
Cryptos = np.unique(data.Instrument[data.Type=='Crypto'])

In [20]:
Report2020 = Report[Report['Sell Date'] <= datetime(2020,12,31,23,59,59)].reset_index(drop=True)
Report2021 = Report[Report['Sell Date'] > datetime(2020,12,31,23,59,59)].reset_index(drop=True)

In [22]:
Report_Stock2020 = Report2020[Report2020.Stock.isin(Stocks)].reset_index(drop=True)
Report_Crypto2020 = Report2020[Report2020.Stock.isin(Cryptos)].reset_index(drop=True)

In [23]:
Report_Stock2021 = Report2021[Report2021.Stock.isin(Stocks)].reset_index(drop=True)
Report_Crypto2021 = Report2021[Report2021.Stock.isin(Cryptos)].reset_index(drop=True)


In [24]:
Report_Stock2020.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Stock Report 2020.csv')
Report_Crypto2020.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Crypto Report 2020.csv')

Report_Stock2021.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Stock Report 2021.csv')
Report_Crypto2021.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Crypto Report 2021.csv')


# Crypto.com

In [60]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\crypto_transactions_record_20220529_232110.csv')

In [62]:
USDEUR = []
for i, time in enumerate(data['Timestamp (UTC)']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%Y-%m-%d %H:%M:%S')))
    print(i, end='\r')

In [ ]:
data['Native Amount (EUR)'] = np.array(USDEUR) * data['Native Amount (USD)']

In [61]:
Cashbacks = data[data['Transaction Description'].isin(['CRO Stake Rewards', 'Card Cashback', 'Card Cashback Reversal', 'Card Rebate Reversal: Netflix', 
'Card Rebate: Netflix', 'Card Rebate: Spotify', 'Referral Bonus Reward', 'Sign-up Bonus Unlocked', 'Supercharger Reward'])].reset_index(drop=True)

Cashbacks = Cashbacks.drop(['To Currency', 'To Amount'],axis=1)
Cashbacks



,Timestamp (UTC),Transaction Description,Currency,Amount,Native Currency,Native Amount,Native Amount (in USD),Transaction Kind,Transaction Hash
0,2021-12-30 18:52:27,Card Cashback,CRO,2.278596,EUR,1.14,1.230807,referral_card_cashback,NaN
1,2021-12-30 17:32:37,Card Cashback,CRO,0.424724,EUR,0.21,0.226728,referral_card_cashback,NaN
2,2021-12-30 09:29:00,Card Cashback,CRO,0.557017,EUR,0.28,0.302304,referral_card_cashback,NaN
3,2021-12-30 08:17:06,Card Cashback,CRO,0.338780,EUR,0.17,0.183541,referral_card_cashback,NaN
4,2021-12-29 19:07:12,Card Cashback,CRO,0.642344,EUR,0.32,0.345490,referral_card_cashback,NaN
...,...,...,...,...,...,...,...,...,...
192,2021-06-11 09:45:08,Card Rebate: Spotify,CRO,100.868755,EUR,9.99,10.785760,reimbursement,NaN
193,2021-06-03 23:25:12,Card Cashback,CRO,1.797610,EUR,0.20,0.215931,referral_card_cashback,NaN
194,2021-05-13 06:20:10,Card Cashback Reversal,CRO,-0.119895,EUR,0.02,0.021593,card_cashback_reverted,NaN
195,2021-05-11 09:45:21,Card Cashback,CRO,0.119895,EUR,0.02,0.021593,referral_card_cashback,NaN


In [59]:
sum(Cashbacks['Native Amount (EUR)'])

KeyError: 'Native Amount (EUR)'

In [ ]:
Cashbacks.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Crypto Cashback.csv')

In [ ]:
data = data[np.logical_not(data['Transaction Description'].isin(['From +358504077653', 'Supercharger Deposit (via app)']))].reset_index(drop=True)

data1 = data[['Timestamp (UTC)', 'Transaction Description', 'Currency', 'Amount', 'Native Amount (EUR)']].reset_index(drop=True)
data1.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']

data2 = data[['Timestamp (UTC)', 'Transaction Description', 'To Currency', 'To Amount', 'Native Amount (EUR)']]
data2.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']
data2 = data2[np.logical_not(data2.Change.isna())].reset_index(drop=True)

data = pd.concat([data1, data2], axis=0).reset_index(drop=True)

data["UTC_Time"] = pd.to_datetime(data["UTC_Time"], format='%m/%d/%Y %H:%M')
data = data.sort_values(by="UTC_Time")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')


data

In [ ]:
uniqueCoins = np.unique(data.Coin)
uniqueCoins

In [ ]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': data['Change'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'date_buy': data['UTC_Time'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'price_buy': data['Total Price (EUR)'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'amount_sell': data['Change'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'date_sell': data['UTC_Time'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'price_sell': (-data['Total Price (EUR)'][(data.Coin==coin) & (data.Change<0)]).to_list()}

In [ ]:
ALL

In [ ]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [ ]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [ ]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [ ]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Crypto Report.csv')

In [ ]:
ALL

In [ ]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

In [ ]:
260.80-219.15+92.64

In [ ]:
3022.4-1194.71+15.93 - 75.48 


In [53]:
sum(profit) + sum(Cashback['Total Price (EUR)']) -sum(loss)-sum(Fee['Total Price (EUR)']) - sum(Deposits['Fee (EUR)'])

1771.820000000013

In [54]:
sum(Withdraw['Total Price (EUR)'])-sum(Deposits['Total Price (EUR)'])

1607.670000000001

In [36]:
sum(Deposits['Total Price (EUR)'])

10418.68

In [37]:
sum(loss)

1198.8099999999986

In [38]:
sum(Fee['Total Price (EUR)']) + sum(Deposits['Fee (EUR)'])

80.84

In [348]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

ADA :    0.00000
ALICE :    0.00000
BAR :    0.00000
BNB :    0.00285
BTC :    0.00000
BUSD :    0.15344
DOGE :    0.00000
DOT :    0.00000
ENJ :    0.00000
ETH :    0.00000
EUR :    0.00031
FXS :    0.00000
HNT :    0.00000
IOTA :    0.00000
LTC :    0.00000
LUNA :    0.00000
MANA :    0.00000
MATIC :    0.00000
MTL :    0.00000
NFT :    18334.49575
RVN :    0.00000
SFP :    0.00000
SHIB :    0.00000
SOL :    0.00000
STMX :    0.00000
TRX :    0.00000
USDT :    0.00000
WABI :    0.14720
XLM :    0.00000
XRP :    0.00000
ZRX :    0.00000


# Etoro

In [161]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro.csv')

In [162]:
data

,Position ID,Action,Amount,Units,Open Date,Close Date,Leverage,Spread,Profit,Open Rate,Close Rate,Take profit rate,Stop lose rate,Rollover Fees and Dividends,Copied From,Type,ISIN,Notes
0,798900619,Buy Boeing,100.00,0.465875,18/11/2020 15:33:00,01/12/2020 14:49:07,1,0.0,0.22,214.63,215.10,2361.13,0.01,0.0,-,Stocks,US0970231058,NaN
1,798046203,Buy Boeing,100.00,0.478515,17/11/2020 20:20:54,01/12/2020 14:39:45,1,0.0,2.95,208.98,215.14,2298.78,0.01,0.0,-,Stocks,US0970231058,NaN
2,673064189,Buy Amazon.com Inc,333.74,0.099928,13/07/2020 15:05:11,01/12/2020 14:36:05,1,0.0,-17.11,3339.84,3168.62,0.00,0.01,0.0,-,Stocks,US0231351067,NaN
3,799341020,Buy Boeing,50.00,0.243605,18/11/2020 20:32:06,01/12/2020 14:35:17,1,0.0,1.93,205.25,213.19,2257.75,0.01,0.0,-,Stocks,US0970231058,NaN
4,800480812,Buy Boeing,50.00,0.246427,19/11/2020 17:16:56,01/12/2020 14:35:09,1,0.0,2.54,202.90,213.19,2231.90,0.01,0.0,-,Stocks,US0970231058,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,804049275,"Buy Tesla Motors, Inc.",58.28,0.111948,23/11/2020 15:24:57,11/01/2021 20:18:50,1,0.0,32.74,520.60,813.06,5725.50,0.01,0.0,-,Stocks,US88160R1014,NaN
415,804055078,"Buy Tesla Motors, Inc.",50.00,0.096028,23/11/2020 15:26:45,11/01/2021 20:18:50,1,0.0,28.08,520.68,813.06,5727.49,0.01,0.0,-,Stocks,US88160R1014,NaN
416,798030691,"Buy Tesla Motors, Inc.",100.00,0.227433,17/11/2020 20:10:38,11/01/2021 20:18:50,1,0.0,84.92,439.69,813.06,4836.59,0.01,0.0,-,Stocks,US88160R1014,NaN
417,797571688,"Buy Tesla Motors, Inc.",100.00,0.224800,17/11/2020 16:05:11,11/01/2021 20:18:50,1,0.0,82.78,444.84,813.06,4893.24,0.01,0.0,-,Stocks,US88160R1014,NaN


In [158]:
USDEUR = []
for i, time in enumerate(data['Open Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [164]:
data['Amount (EUR)'] = np.round(USDEUR * data['Amount'],2)

In [168]:
USDEUR = []
for i, time in enumerate(data['Close Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [173]:
data['Amount Sell (EUR)'] = np.round((data['Amount'] + data['Profit']) * USDEUR,2)


data['Profit (EUR)'] = data['Amount Sell (EUR)'] - data['Amount (EUR)']

data['Action'] = [stock.replace('Buy ','') for stock in data['Action']]

In [180]:
data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro EUR.csv')

In [198]:
data_buy = data[['Action', 'Units', 'Open Date', 'Type', 'Amount (EUR)', 'Amount']]
data_buy.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)', 'Total Price (USD)']
data_buy['Action'] = ['Buy']*len(data_buy.Instrument)


data_sell = data[['Action', 'Units', 'Close Date', 'Type', 'Amount Sell (EUR)']]
data_sell.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)']
data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
data_sell['Action'] = ['Sell']*len(data_sell.Instrument)

data = pd.concat([data_buy, data_sell], axis=0)

data = data[['Type', 'Action', 'Instrument', 'Units', 'Date', 'Total Price (EUR)', 'Total Price (USD)']]

C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_buy['Action'] = ['Buy']*len(data_buy.Instrument)
C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [200]:
data["Date"] = pd.to_datetime(data["Date"], format='%d/%m/%Y %H:%M:%S')
data = data.sort_values(by="Date")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')

In [201]:
uniqueInstruments = np.unique(data.Instrument)
uniqueInstruments

array(['AIRBUS SE', 'Abbott Laboratories', 'Advanced Micro Devices Inc',
       'Air France-KLM', 'Alibaba', 'Alphabet', 'Amazon.com Inc',
       'American Airlines Group Inc', 'Apple', 'Aurora Cannabis Inc',
       'Ballard Power Systems Inc.', 'BioNano Genomics Inc', 'Bitcoin',
       'Boeing', 'Booking Holdings Inc', 'Boxlight Corp',
       'Build and Build', 'Cardano', 'Cocrystal Pharma Inc',
       'Delta Air Lines Inc (DE)', 'Dogecoin', 'Ethereum',
       'FuelCell Energy Inc', 'Globalstar', 'IOTA', 'Litecoin',
       'Meta Platforms Inc', 'Microsoft', 'Moderna Inc',
       'NVIDIA Corporation', 'Nio Inc.', 'Palantir Technologies Inc.',
       'Pfizer', 'Plug Power Inc', 'Tesla Motors, Inc.', 'Walmart Inc.',
       'Zoom Video Communications Inc', 'easyJet'], dtype=object)

In [228]:
ALL ={}

for instrument in uniqueInstruments:
    ALL[instrument] ={'amount_buy': data['Units'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'date_buy': data['Date'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'price_buy': data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'amount_sell': (-data['Units'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list(),
                      'date_sell': data['Date'][(data.Instrument==instrument) & (data.Action == 'Sell')].to_list(),
                      'price_sell': (-data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list()}

In [229]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
stock = []
profit = []
loss = []
result = []
for instrument in uniqueInstruments:
    while (len(All_copy[instrument]['amount_buy']) != 0) and (len(All_copy[instrument]['amount_sell']) != 0):
        buy_date.append(All_copy[instrument]['date_buy'][0])
        sell_date.append(All_copy[instrument]['date_sell'][0])
        stock.append(instrument)
        if All_copy[instrument]['amount_buy'][0] <= abs(All_copy[instrument]['amount_sell'][0]):
            
            amount.append(All_copy[instrument]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[instrument]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[instrument]['price_sell'][0] * 
                                All_copy[instrument]['amount_buy'][0] / All_copy[instrument]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[instrument]['amount_sell'][0] += All_copy[instrument]['amount_buy'][0]
            All_copy[instrument]['price_sell'][0] += sellPricePortion
            
            if All_copy[instrument]['amount_sell'][0] == 0:
                del All_copy[instrument]['amount_sell'][0]
                del All_copy[instrument]['price_sell'][0]
                del All_copy[instrument]['date_sell'][0]
            del All_copy[instrument]['amount_buy'][0]
            del All_copy[instrument]['price_buy'][0]
            del All_copy[instrument]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[instrument]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[instrument]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[instrument]['price_buy'][0] * 
                                All_copy[instrument]['amount_sell'][0] / All_copy[instrument]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[instrument]['amount_buy'][0] += All_copy[instrument]['amount_sell'][0]
            All_copy[instrument]['price_buy'][0] -= buyPricePortion
            
            if All_copy[instrument]['amount_buy'][0] == 0:
                del All_copy[instrument]['amount_buy'][0]
                del All_copy[instrument]['price_buy'][0]
                del All_copy[instrument]['date_buy'][0]
            del All_copy[instrument]['amount_sell'][0]
            del All_copy[instrument]['price_sell'][0]
            del All_copy[instrument]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = abs(result *(result>=0))
loss = abs(result * (result<0))

In [230]:
Report =pd.DataFrame({'Stock': stock,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [231]:
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [260]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report.csv')

In [245]:
Stocks = np.unique(data.Instrument[data.Type=='Stocks'])
Cryptos = np.unique(data.Instrument[data.Type=='Crypto'])

In [256]:
Report2020 = Report[Report['Sell Date'] <= datetime(2020,12,31,23,59,59)].reset_index(drop=True)
Report2021 = Report[Report['Sell Date'] > datetime(2020,12,31,23,59,59)].reset_index(drop=True)

In [257]:
Report_Stock = Report2021[Report2021.Stock.isin(Stocks)].reset_index(drop=True)

In [258]:
Report_Crypto = Report2021[Report2021.Stock.isin(Cryptos)].reset_index(drop=True)


In [259]:
Report2020.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report 2020.csv')

Report_Stock.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Stock Report 2021.csv')
Report_Crypto.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Crypto Report 2021.csv')


In [65]:
get_rate("USD", "EUR",datetime(2021,4,20))

0.8298066550493735

In [66]:
0.829806655049373*25

20.745166376234327

In [368]:
datetime(2021,5,16).day

16

In [367]:
2225.57 + 2.76 - 1104.95 - 27.92 

1095.4600000000003

# Crypto.com

In [117]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\crypto_transactions_record_20220529_232110.csv')

In [72]:
Cashbacks = data[data['Transaction Description'].isin(['CRO Stake Rewards', 'Card Cashback', 'Card Cashback Reversal', 'Card Rebate Reversal: Netflix', 
'Card Rebate: Netflix', 'Card Rebate: Spotify', 'Referral Bonus Reward', 'Sign-up Bonus Unlocked', 'Supercharger Reward'])].reset_index(drop=True)

Cashbacks = Cashbacks.drop(['To Currency', 'To Amount', 'Native Currency', 'Native Amount (in USD)', 'Transaction Kind', 'Transaction Hash'],axis=1)
Cashbacks



,Timestamp (UTC),Transaction Description,Currency,Amount,Native Amount (EUR)
0,30-12-2021 18:52,Card Cashback,CRO,2.278596,1.14
1,30-12-2021 17:32,Card Cashback,CRO,0.424724,0.21
2,30-12-2021 09:29,Card Cashback,CRO,0.557017,0.28
3,30-12-2021 08:17,Card Cashback,CRO,0.338780,0.17
4,29-12-2021 19:07,Card Cashback,CRO,0.642344,0.32
...,...,...,...,...,...
192,11-06-2021 09:45,Card Rebate: Spotify,CRO,100.868755,9.99
193,03-06-2021 23:25,Card Cashback,CRO,1.797610,0.20
194,13-05-2021 06:20,Card Cashback Reversal,CRO,-0.119895,0.02
195,11-05-2021 09:45,Card Cashback,CRO,0.119895,0.02


In [73]:
sum(Cashbacks['Native Amount (EUR)'])

227.31657899000012

In [74]:
Cashbacks.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Crypto Cashback.csv')

In [79]:
data

,Timestamp (UTC),Transaction Description,Currency,Amount,To Currency,To Amount,Native Currency,Native Amount (EUR),Native Amount (in USD),Transaction Kind,Transaction Hash
0,30-12-2021 18:52,Card Cashback,CRO,2.278596,NaN,NaN,EUR,1.14,1.230807,referral_card_cashback,NaN
1,30-12-2021 17:32,Card Cashback,CRO,0.424724,NaN,NaN,EUR,0.21,0.226728,referral_card_cashback,NaN
2,30-12-2021 09:29,Card Cashback,CRO,0.557017,NaN,NaN,EUR,0.28,0.302304,referral_card_cashback,NaN
3,30-12-2021 08:17,Card Cashback,CRO,0.338780,NaN,NaN,EUR,0.17,0.183541,referral_card_cashback,NaN
4,29-12-2021 19:07,Card Cashback,CRO,0.642344,NaN,NaN,EUR,0.32,0.345490,referral_card_cashback,NaN
...,...,...,...,...,...,...,...,...,...,...,...
209,11-05-2021 09:45,Card Cashback,CRO,0.119895,NaN,NaN,EUR,0.02,0.021593,referral_card_cashback,NaN
210,11-05-2021 09:07,Buy SHIB,SHIB,1000000.000000,NaN,NaN,EUR,28.55,30.824168,crypto_purchase,NaN
211,20-04-2021 18:54,Sign-up Bonus Unlocked,CRO,351.146656,NaN,NaN,USD,20.75,25.000000,referral_gift,NaN
212,20-04-2021 18:53,CRO Stake,CRO,-2149.000000,NaN,NaN,EUR,350.11,377.998231,lockup_lock,NaN


In [118]:
# data = data[np.logical_not(data['Transaction Description'].isin(['From +358504077653', 'Supercharger Deposit (via app)']))].reset_index(drop=True)

data1 = data[['Timestamp (UTC)', 'Transaction Description', 'Currency', 'Amount', 'Native Amount (EUR)']].reset_index(drop=True)
data1.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']

data2 = data[['Timestamp (UTC)', 'Transaction Description', 'To Currency', 'To Amount', 'Native Amount (EUR)']]
data2.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']
data2 = data2[np.logical_not(data2.Change.isna())].reset_index(drop=True)

data = pd.concat([data1, data2], axis=0).reset_index(drop=True)
data = data[np.logical_not(data['Transaction Description'].isin(['CRO Stake', 'Supercharger Deposit (via app)', 
                                                                 'Supercharger Withdrawal (via app)', 'Card Cashback Reversal',
                                                                 'Supercharger Stake (via app)']))].reset_index(drop=True)

data["UTC_Time"] = pd.to_datetime(data["UTC_Time"], format='%d-%m-%Y %H:%M').reset_index(drop=True)
data = data.sort_values(by="UTC_Time")

data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Etoro Buy Sell EUR.csv')


data

,UTC_Time,Transaction Description,Coin,Change,Total Price (EUR)
204,2021-04-20 18:44:00,EUR -> CRO,CRO,2150.170000,350.00
202,2021-04-20 18:44:00,EUR -> CRO,EUR,-350.000000,350.00
201,2021-04-20 18:54:00,Sign-up Bonus Unlocked,CRO,351.146656,20.75
200,2021-05-11 09:07:00,Buy SHIB,SHIB,1000000.000000,28.55
199,2021-05-11 09:45:00,Card Cashback,CRO,0.119895,0.02
...,...,...,...,...,...
4,2021-12-29 19:07:00,Card Cashback,CRO,0.642344,0.32
3,2021-12-30 08:17:00,Card Cashback,CRO,0.338780,0.17
2,2021-12-30 09:29:00,Card Cashback,CRO,0.557017,0.28
1,2021-12-30 17:32:00,Card Cashback,CRO,0.424724,0.21


In [119]:
uniqueCoins = np.unique(data.Coin)
uniqueCoins

array(['AXS', 'CRO', 'DYDX', 'EUR', 'HBAR', 'SDN', 'SHIB'], dtype=object)

In [120]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': data['Change'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'date_buy': data['UTC_Time'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'price_buy': data['Total Price (EUR)'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'amount_sell': data['Change'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'date_sell': data['UTC_Time'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'price_sell': (-data['Total Price (EUR)'][(data.Coin==coin) & (data.Change<0)]).to_list()}

In [121]:
ALL

{'AXS': {'amount_buy': [0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415],
  'date_buy': [Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-07 03:19:00'),
   Timestamp('2021-10-07 23:21:00'),
   Timestamp('2021-10-09 01:40:00'),
   Timestamp('2021-10-10 01:18:00'),
   Timestamp('2021-10-11 01:14:00'),
   Timestamp('2021-10-12 01:02:00'),
   Timestamp('2021-10-13 04:18:00'),
   Timestamp('2021-10-14 01:43:00'),
   Timestamp('2021-10-15 01:02:00'),
   Timestamp('2021-10-16 01:02:00'),
   Timestamp('2021-10-17 01:02:00')

In [122]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [123]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [124]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [126]:
Report.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Crypto Report.csv')

In [127]:
ALL

{'AXS': {'amount_buy': [0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415],
  'date_buy': [Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-07 03:19:00'),
   Timestamp('2021-10-07 23:21:00'),
   Timestamp('2021-10-09 01:40:00'),
   Timestamp('2021-10-10 01:18:00'),
   Timestamp('2021-10-11 01:14:00'),
   Timestamp('2021-10-12 01:02:00'),
   Timestamp('2021-10-13 04:18:00'),
   Timestamp('2021-10-14 01:43:00'),
   Timestamp('2021-10-15 01:02:00'),
   Timestamp('2021-10-16 01:02:00'),
   Timestamp('2021-10-17 01:02:00')

In [128]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

AXS :    0.04632
CRO :    5788.37114
DYDX :    0.22285
EUR :    0.00000
HBAR :    3.91069
SDN :    0.29616
SHIB :    2000000.00000


In [129]:
260.80-219.15+92.64

134.29000000000002

In [56]:
3022.4-1194.71+15.93 - 75.48 


1768.14

In [1]:
from datetime import datetime
import pandas as pd
import requests
import numpy as np
from forex_python.converter import get_rate

In [ ]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\part-00000-67beda92-080e-4801-bee9-784943534377-c000.csv')

In [ ]:
timestamps1 = [int(datetime.strptime(time, '%Y-%m-%d %H:%M:%S').timestamp()*1000) for time in data.UTC_Time]
timestamps2 = [time + 59999 for time in timestamps1]

In [ ]:
i = 10
tick_interval = '1m'

prices = []
markets = []
markets1 = []
prices1 = []
for i in range(len(timestamps1)):
    price1 = '1'
    market1 = 'None'
    print(i, end='\r')
    if data.Coin[i] not in ['EUR', 'NFT']:
        try:
            market = data.Coin[i]+'EUR'
            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
            price = requests.get(url).json()[0][1]
            
        except:
            try:
                market = 'EUR'+data.Coin[i]
                url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                price = str(1/float(requests.get(url).json()[0][1]))
                market = data.Coin[i]+'EUR'
                
            except:
                try:
                    market = data.Coin[i]+'USDT'
                    market1 = 'EURUSDT'
                    url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                    price = requests.get(url).json()[0][1]
                    
                    url1 = 'https://api.binance.com/api/v3/klines?symbol=EURUSDT&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                    price1 = requests.get(url1).json()[0][1]
                
                except:
                    try:
                        market = data.Coin[i]+'BUSD'
                        market1 = 'EURBUSD'
                        url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                        price = requests.get(url).json()[0][1]

                        url1 = 'https://api.binance.com/api/v3/klines?symbol=EURBUSD&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                        price1 = requests.get(url1).json()[0][1]
                        
                    except:
                        try:
                            market = data.Coin[i] + 'BTC'
                            market1 = 'EURBTC'
                            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                            price = requests.get(url).json()[0][1]

                            url1 = 'https://api.binance.com/api/v3/klines?symbol=BTCEUR&interval='+tick_interval+'&startTime='+str(timestamps1[i])+'&endTime='+str(timestamps2[i])
                            price1 = str(1/float(requests.get(url1).json()[0][1]))
                            
                        except:
                            market = data.Coin[i]+'USDT'
                            market1 = 'EURUSDT'
                            url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval+'&startTime='+str(timestamps1[i])
                            price = requests.get(url).json()[0][1]

                            url1 = 'https://api.binance.com/api/v3/klines?symbol=EURUSDT&interval='+tick_interval+'&startTime='+str(requests.get(url).json()[0][0])
                            price1 = requests.get(url1).json()[0][1]

                            print(requests.get(url).json()[0][0])
                            print(requests.get(url1).json()[0][0])
                            print(data.Coin[i])
                            
    elif data.Coin[i] == 'EUR':
        price = '1'
        market = 'EUR'
    elif data.Coin[i] == 'NFT':
        price = '0'
        market = 'NFT'
    prices.append(price)
    prices1.append(price1)
    markets.append(market)
    markets1.append(market1)

In [ ]:
UnitPrices = [float(prices[i]) / float(prices1[i]) for i in range(len(prices))]
totalPrices = [UnitPrices[i] * float(data.Change[i]) for i in range(len(prices)) ]

In [ ]:
data1 = pd.DataFrame({'Unit Prices': UnitPrices, 'Total Price (EUR)': totalPrices})
#'Tool':markets, 'Tool Price':prices, 'Tool to EUR': markets1, 
#                       'Coefficient Price': prices1, 

In [ ]:
Data = pd.concat([data, data1], axis=1)

In [ ]:
Data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_EUR.csv')

In [2]:
Data=pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_EUR.csv')

In [3]:
uniqueCoins = np.unique(Data.Coin)
uniqueCoins

array(['ADA', 'ALICE', 'BAR', 'BAT', 'BNB', 'BUSD', 'CRV', 'DOGE', 'ETH',
       'EUR', 'IOTA', 'LUNA', 'MANA', 'QI', 'SFP', 'SHIB', 'USDT', 'WABI'],
      dtype=object)

Calculate the spot by the end of the year

In [5]:
for i in uniqueCoins:
    print(i +':   ', np.round(np.sum(Data.Change[Data.Coin==i]),10))

ADA:    95.15150236
ALICE:    0.0
BAR:    0.0
BAT:    0.0
BNB:    1.58643072
BUSD:    0.0
CRV:    17.5
DOGE:    0.2767986
ETH:    0.0
EUR:    -0.0
IOTA:    203.833
LUNA:    0.0
MANA:    70.0
QI:    7.36098866
SFP:    0.0
SHIB:    5185519.199999999
USDT:    2.11259859
WABI:    0.0


In [6]:
for i in uniqueCoins:
    print(i +':   ', np.round(np.sum(Data['Total Price (EUR)'][Data.Coin==i]),10))

ADA:    143.2549866151
ALICE:    0.0052673485
BAR:    0.0
BAT:    -51.6147632859
BNB:    220.7878373527
BUSD:    -0.1187395808
CRV:    92.6828724353
DOGE:    49.3721849666
ETH:    -106.2022101428
EUR:    -0.0
IOTA:    165.3203980797
LUNA:    -6.3964919372
MANA:    180.6271289556
QI:    1.1015426393
SFP:    -27.7216191139
SHIB:    224.5338609278
USDT:    -1.854270411
WABI:    111.1158575715


In [8]:
Data["UTC_Time"] = pd.to_datetime(Data["UTC_Time"], format='%Y-%m-%d %H:%M:%S')
Data = Data.sort_values(by="UTC_Time")

Data

,Unnamed: 0,User_ID,UTC_Time,Account,Operation,Coin,Change,Remark,Unit Prices,Total Price (EUR)
0,0,71190841,2021-02-08 08:33:27,Spot,Deposit,EUR,100.000000,NaN,1.000000,100.000000
1,1,71190841,2021-02-08 08:44:50,Spot,Buy,DOGE,418.000000,NaN,0.058370,24.398660
2,2,71190841,2021-02-08 08:44:50,Spot,Transaction Related,EUR,-24.871000,NaN,1.000000,-24.871000
3,3,71190841,2021-02-08 08:44:53,Spot,Transaction Related,EUR,-0.047600,NaN,1.000000,-0.047600
4,4,71190841,2021-02-08 08:44:53,Spot,Buy,DOGE,0.800000,NaN,0.058370,0.046696
...,...,...,...,...,...,...,...,...,...,...
726,726,71190841,2021-12-14 14:53:50,Spot,Fee,BNB,-0.000075,NaN,459.300000,-0.034544
729,729,71190841,2021-12-16 00:16:24,Spot,Launchpool Interest,QI,7.360989,NaN,0.149646,1.101543
731,731,71190841,2021-12-19 01:08:23,Spot,Buy,USDT,-49.938000,NaN,0.890472,-44.468388
730,730,71190841,2021-12-19 01:08:23,Spot,Buy,MANA,14.000000,NaN,3.108994,43.525913


In [9]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': Data['Change'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'date_buy': Data['UTC_Time'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'price_buy': Data['Total Price (EUR)'][(Data.Coin==coin) & (Data.Change>0)].to_list(),
                'amount_sell': Data['Change'][(Data.Coin==coin) & (Data.Change<0)].to_list(),
                'date_sell': Data['UTC_Time'][(Data.Coin==coin) & (Data.Change<0)].to_list(),
                'price_sell': Data['Total Price (EUR)'][(Data.Coin==coin) & (Data.Change<0)].to_list()}

In [10]:
ALL

{'ADA': {'amount_buy': [27.5,
   67.52,
   49.51,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   57.4,
   42.6,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   104.6,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   0.02497654,
   117.03,
   127.7,
 

In [11]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [12]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [13]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%Y-%m-%d %H:%M:%S')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%Y-%m-%d %H:%M:%S')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)
Report = Report[Report['Sell Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

In [14]:
Report.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Report.csv')

In [15]:
Deposits = Data[Data.Operation == 'Deposit'].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Deposits['Total Price (EUR)'] = np.round(Deposits['Total Price (EUR)'], 2)
Deposits['Fee (EUR)'] = [0.0]*len(Deposits['Total Price (EUR)'])
# Deposits.iloc[[8, 9, 13, 14], 6] = [0.2, 4.4, 5.57, 13.26]
Deposits.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)', 'Fee (EUR)']
Deposits["Date"] = pd.to_datetime(Deposits["Date"], format='%Y-%m-%d %H:%M:%S')
Deposits = Deposits.sort_values(by="Date").reset_index(drop=True)
Deposits = Deposits[Deposits['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)
Deposits

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR),Fee (EUR)
0,2021-02-08 08:33:27,Deposit,EUR,100.0,1.0,100.0,0.0
1,2021-02-16 09:45:09,Deposit,EUR,100.0,1.0,100.0,0.0
2,2021-05-13 23:27:09,Deposit,EUR,98.2,1.0,98.2,0.0
3,2021-05-19 07:20:29,Deposit,EUR,196.4,1.0,196.4,0.0
4,2021-05-19 16:10:23,Deposit,EUR,196.4,1.0,196.4,0.0
5,2021-05-23 11:35:19,Deposit,EUR,196.4,1.0,196.4,0.0
6,2021-06-22 15:28:54,Deposit,EUR,196.4,1.0,196.4,0.0


In [16]:
Deposits.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Deposits.csv')

In [17]:
Withdraw = Data[Data.Operation.isin(['Withdraw', 'Binance Card Spending'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Unnamed: 0'],axis=1)
Withdraw = Withdraw.fillna('No Fee')
Withdraw['Total Price (EUR)'] = -np.round(Withdraw['Total Price (EUR)'], 2)
Withdraw['Change'] *= -1
Withdraw.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Fee (EUR)', 'Unit Price (EUR)', 'Total Price (EUR)']
Withdraw = Withdraw [['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)', 'Fee (EUR)']]
Withdraw["Date"] = pd.to_datetime(Withdraw["Date"], format='%Y-%m-%d %H:%M:%S')
Withdraw = Withdraw.sort_values(by="Date").reset_index(drop=True)
Withdraw = Withdraw[Withdraw['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

Withdraw

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR),Fee (EUR)


In [19]:
Withdraw.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Withdrawals.csv')

In [20]:
Fee = Data[Data.Operation.isin(['Fee'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Fee['Total Price (EUR)'] = -np.round(Fee['Total Price (EUR)'], 2)
Fee['Change'] *= -1
Fee.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)']
Fee["Date"] = pd.to_datetime(Fee["Date"], format='%Y-%m-%d %H:%M:%S')
Fee = Fee.sort_values(by="Date").reset_index(drop=True)
Fee = Fee[Fee['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)


Fee

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR)
0,2021-02-08 14:36:19,Fee,SFP,0.021230,1.060316,0.02
1,2021-02-08 16:41:00,Fee,LUNA,0.004010,4.058095,0.02
2,2021-02-08 21:57:50,Fee,ADA,0.027500,0.589000,0.02
3,2021-02-18 07:59:28,Fee,BNB,0.000133,147.700000,0.02
4,2021-02-18 08:02:12,Fee,BNB,0.000131,147.285800,0.02
...,...,...,...,...,...,...
154,2021-11-26 14:01:54,Fee,BNB,0.000285,523.700000,0.15
155,2021-12-14 11:48:25,Fee,BNB,0.000051,462.400000,0.02
156,2021-12-14 11:48:25,Fee,BNB,0.000031,462.400000,0.01
157,2021-12-14 14:53:50,Fee,BNB,0.000075,459.300000,0.03


In [21]:
Fee.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Fees.csv')

In [22]:
Cashback = Data[Data.Operation.isin(['Card Cashback', 'Commission Rebate', 'Launchpool Interest'])].reset_index(drop=True).drop(['User_ID', 'Account', 'Remark', 'Unnamed: 0'],axis=1)
Cashback['Total Price (EUR)'] = np.round(Cashback['Total Price (EUR)'], 2)
Cashback['Change'] *= -1
Cashback.columns = ['Date', 'Operation', 'Coin', 'Amount', 'Unit Price (EUR)', 'Total Price (EUR)']
Cashback["Date"] = pd.to_datetime(Cashback["Date"], format='%Y-%m-%d %H:%M:%S')
Cashback = Cashback.sort_values(by="Date").reset_index(drop=True)
Cashback = Cashback[Cashback['Date'] <= datetime(2021,12,31,23,59,59)].reset_index(drop=True)

Cashback

,Date,Operation,Coin,Amount,Unit Price (EUR),Total Price (EUR)
0,2021-04-09 00:30:07,Launchpool Interest,ALICE,-0.177064,10.727387,1.9
1,2021-12-16 00:16:24,Launchpool Interest,QI,-7.360989,0.149646,1.1


In [23]:
Cashback.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Binance\Binance_Cashbacks.csv')

In [ ]:
np.unique(Data.Operation)

In [24]:
sum(profit) + sum(Cashback['Total Price (EUR)']) -sum(loss)-sum(Fee['Total Price (EUR)']) - sum(Deposits['Fee (EUR)'])

840.3699999999974

In [25]:
sum(Withdraw['Total Price (EUR)'])-sum(Deposits['Total Price (EUR)'])

-1083.8

In [26]:
sum(Deposits['Total Price (EUR)'])

1083.8

In [27]:
sum(loss)

567.0999999999998

In [28]:
sum(Fee['Total Price (EUR)']) + sum(Deposits['Fee (EUR)'])

15.419999999999986

In [29]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

ADA :    95.15150
ALICE :    0.00000
BAR :    0.00000
BAT :    0.00000
BNB :    1.58643
BUSD :    0.00000
CRV :    17.50000
DOGE :    0.27680
ETH :    0.00000
EUR :    0.00000
IOTA :    203.83300
LUNA :    0.00000
MANA :    70.00000
QI :    7.36099
SFP :    0.00000
SHIB :    5185519.20000
USDT :    2.11260
WABI :    0.00000


# Etoro

In [36]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro.csv')

In [37]:
data

,Position ID,Action,Amount,Units,Open Date,Close Date,Leverage,Spread,Profit,Open Rate,Close Rate,Take profit rate,Stop lose rate,Rollover Fees and Dividends,Copied From,Type,ISIN,Notes
0,793962426,Buy Xiaomi Corp,10.01,3.331238,13/11/2020 01:30:11,30/12/2020 01:30:20,1,0.00,4.06,23.30,32.75,248.56,0.01,0.0,LG2001,Stocks,KYG9830T1067,NaN
1,816766394,Buy Xiaomi Corp,3.52,1.123064,02/12/2020 07:04:09,30/12/2020 01:30:20,1,0.00,1.22,24.30,32.75,267.28,0.01,0.0,LG2001,Stocks,KYG9830T1067,NaN
2,847419499,Buy Nano Dimension Ltd,1.29,0.123682,23/12/2020 17:18:37,29/12/2020 14:35:45,1,0.00,-0.27,10.43,8.24,0.00,0.01,0.0,LG2001,Stocks,US63008G2030,NaN
3,793480168,Buy Alibaba,6.01,0.022577,12/11/2020 16:52:39,29/12/2020 14:35:20,1,0.00,-0.82,266.20,230.09,"2,928.42",0.00,0.0,LG2001,Stocks,US01609W1027,NaN
4,793480163,Buy Alibaba,2.00,0.007513,12/11/2020 16:52:39,29/12/2020 14:35:20,1,0.00,-0.27,266.20,230.09,"2,937.44",0.00,0.0,LG2001,Stocks,US01609W1027,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,800952174,Buy Aston Martin Lagonda Global Holdings PLC,120.98,6.247181,20/11/2020 09:43:09,12/01/2021 14:34:02,1,0.00,19.83,"1,446.00","1,679.40",0.00,0.20,0.0,-,Stocks,GB00BN7CG237,NaN
650,801015160,Buy Aston Martin Lagonda Global Holdings PLC,120.00,6.305629,20/11/2020 13:16:43,12/01/2021 14:34:00,1,0.00,21.05,"1,434.00","1,679.40",0.00,0.20,0.0,-,Stocks,GB00BN7CG237,NaN
651,892979086,Buy Ethereum,110.00,0.099325,12/01/2021 11:51:47,12/01/2021 14:16:56,1,1.99,-6.14,"1,107.47","1,045.70",0.00,0.00,0.0,-,Crypto,NaN,NaN
652,892979220,Buy XRP,110.00,369.623656,12/01/2021 11:52:17,12/01/2021 14:16:47,1,2.55,-5.03,0.30,0.28,0.00,0.00,0.0,-,Crypto,NaN,NaN


In [39]:
USDEUR = []
for i, time in enumerate(data['Open Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i, end='\r')

In [40]:
data['Amount (EUR)'] = np.round(USDEUR * data['Amount'],2)

In [41]:
USDEUR = []
for i, time in enumerate(data['Close Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i, end='\r')

In [42]:
data['Amount Sell (EUR)'] = np.round((data['Amount'] + data['Profit']) * USDEUR,2)


data['Profit (EUR)'] = data['Amount Sell (EUR)'] - data['Amount (EUR)']

data['Action'] = [stock.replace('Buy ','') for stock in data['Action']]

In [43]:
data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro EUR.csv')

In [44]:
data_buy = data[['Action', 'Units', 'Open Date', 'Type', 'Amount (EUR)', 'Amount']]
data_buy.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)', 'Total Price (USD)']
data_buy['Action'] = ['Buy']*len(data_buy.Instrument)


data_sell = data[['Action', 'Units', 'Close Date', 'Type', 'Amount Sell (EUR)']]
data_sell.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)']
data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
data_sell['Action'] = ['Sell']*len(data_sell.Instrument)

data = pd.concat([data_buy, data_sell], axis=0)

data = data[['Type', 'Action', 'Instrument', 'Units', 'Date', 'Total Price (EUR)', 'Total Price (USD)']]

C:\Users\JKHATRI\AppData\Local\Temp\ipykernel_15820\18286657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_buy['Action'] = ['Buy']*len(data_buy.Instrument)
C:\Users\JKHATRI\AppData\Local\Temp\ipykernel_15820\18286657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
C:\Users\JKHATRI\AppData\Local\Temp\ipykernel_15820\18286657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [45]:
data["Date"] = pd.to_datetime(data["Date"], format='%d/%m/%Y %H:%M:%S')
data = data.sort_values(by="Date")

data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Buy Sell EUR.csv')

In [46]:
uniqueInstruments = np.unique(data.Instrument)
uniqueInstruments

array(['20+ Year Trsy Bond Ishares', '9F Inc', 'AIRBUS SE',
       'AMC Entertainment Holdings Inc', 'AYRO Inc',
       'Advanced Micro Devices Inc', 'Air France-KLM', 'Airbnb Inc',
       'Alibaba', 'Amazon.com Inc', 'Aphria Inc.', 'Apple', 'Asana',
       'Asensus Surgical Inc.',
       'Aston Martin Lagonda Global Holdings PLC', 'Athenex Inc',
       'Aurora Cannabis Inc', 'Ballard Power Systems Inc.',
       'Barrick Gold', 'Basic Attention Token', 'Bed Bath & Beyond Inc',
       'BioNTech SE', 'BioNano Genomics Inc', 'Bitcoin',
       'Blue Apron Holdings Inc.', 'Boeing', 'Boxlight Corp',
       'Canadian Solar Inc.', 'Cardano', 'Cenntro Electric Group Limited',
       'Cineworld Group PLC', 'Cloudflare', 'Cocrystal Pharma Inc',
       'Coinbase Global Inc', 'Consumer Discret Select Sector SPDR',
       'Corsair Gaming Inc', 'Crowdstrike Holdings', 'Curve',
       'Decentraland', 'Discovery Inc', 'Dogecoin', 'Dutch Bros Inc.',
       'ElringKlinger', 'Enjin', 'Ethereum', 'Evofem B

In [47]:
ALL ={}

for instrument in uniqueInstruments:
    ALL[instrument] ={'amount_buy': data['Units'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'date_buy': data['Date'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'price_buy': data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'amount_sell': (-data['Units'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list(),
                      'date_sell': data['Date'][(data.Instrument==instrument) & (data.Action == 'Sell')].to_list(),
                      'price_sell': (-data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list()}

In [48]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
stock = []
profit = []
loss = []
result = []
for instrument in uniqueInstruments:
    while (len(All_copy[instrument]['amount_buy']) != 0) and (len(All_copy[instrument]['amount_sell']) != 0):
        buy_date.append(All_copy[instrument]['date_buy'][0])
        sell_date.append(All_copy[instrument]['date_sell'][0])
        stock.append(instrument)
        if All_copy[instrument]['amount_buy'][0] <= abs(All_copy[instrument]['amount_sell'][0]):
            
            amount.append(All_copy[instrument]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[instrument]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[instrument]['price_sell'][0] * 
                                All_copy[instrument]['amount_buy'][0] / All_copy[instrument]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[instrument]['amount_sell'][0] += All_copy[instrument]['amount_buy'][0]
            All_copy[instrument]['price_sell'][0] += sellPricePortion
            
            if All_copy[instrument]['amount_sell'][0] == 0:
                del All_copy[instrument]['amount_sell'][0]
                del All_copy[instrument]['price_sell'][0]
                del All_copy[instrument]['date_sell'][0]
            del All_copy[instrument]['amount_buy'][0]
            del All_copy[instrument]['price_buy'][0]
            del All_copy[instrument]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[instrument]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[instrument]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[instrument]['price_buy'][0] * 
                                All_copy[instrument]['amount_sell'][0] / All_copy[instrument]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[instrument]['amount_buy'][0] += All_copy[instrument]['amount_sell'][0]
            All_copy[instrument]['price_buy'][0] -= buyPricePortion
            
            if All_copy[instrument]['amount_buy'][0] == 0:
                del All_copy[instrument]['amount_buy'][0]
                del All_copy[instrument]['price_buy'][0]
                del All_copy[instrument]['date_buy'][0]
            del All_copy[instrument]['amount_sell'][0]
            del All_copy[instrument]['price_sell'][0]
            del All_copy[instrument]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = abs(result *(result>=0))
loss = abs(result * (result<0))

In [49]:
Report =pd.DataFrame({'Stock': stock,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [50]:
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [51]:
Report.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Report.csv')

In [52]:
Stocks = np.unique(data.Instrument[data.Type=='Stocks'])
Cryptos = np.unique(data.Instrument[data.Type=='Crypto'])

In [53]:
Report2020 = Report[Report['Sell Date'] <= datetime(2020,12,31,23,59,59)].reset_index(drop=True)
Report2021 = Report[Report['Sell Date'] > datetime(2020,12,31,23,59,59)].reset_index(drop=True)

In [54]:
Report_Stock = Report2021[Report2021.Stock.isin(Stocks)].reset_index(drop=True)

In [55]:
Report_Crypto = Report2021[Report2021.Stock.isin(Cryptos)].reset_index(drop=True)


In [56]:
Report2020.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Report 2020.csv')

Report_Stock.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Stock Report 2021.csv')
Report_Crypto.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\eToro\Etoro Crypto Report 2021.csv')


# Crypto.com

In [60]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\crypto_transactions_record_20220529_232110.csv')

In [62]:
USDEUR = []
for i, time in enumerate(data['Timestamp (UTC)']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%Y-%m-%d %H:%M:%S')))
    print(i, end='\r')

In [ ]:
data['Native Amount (EUR)'] = np.array(USDEUR) * data['Native Amount (USD)']

In [61]:
Cashbacks = data[data['Transaction Description'].isin(['CRO Stake Rewards', 'Card Cashback', 'Card Cashback Reversal', 'Card Rebate Reversal: Netflix', 
'Card Rebate: Netflix', 'Card Rebate: Spotify', 'Referral Bonus Reward', 'Sign-up Bonus Unlocked', 'Supercharger Reward'])].reset_index(drop=True)

Cashbacks = Cashbacks.drop(['To Currency', 'To Amount'],axis=1)
Cashbacks



,Timestamp (UTC),Transaction Description,Currency,Amount,Native Currency,Native Amount,Native Amount (in USD),Transaction Kind,Transaction Hash
0,2021-12-30 18:52:27,Card Cashback,CRO,2.278596,EUR,1.14,1.230807,referral_card_cashback,NaN
1,2021-12-30 17:32:37,Card Cashback,CRO,0.424724,EUR,0.21,0.226728,referral_card_cashback,NaN
2,2021-12-30 09:29:00,Card Cashback,CRO,0.557017,EUR,0.28,0.302304,referral_card_cashback,NaN
3,2021-12-30 08:17:06,Card Cashback,CRO,0.338780,EUR,0.17,0.183541,referral_card_cashback,NaN
4,2021-12-29 19:07:12,Card Cashback,CRO,0.642344,EUR,0.32,0.345490,referral_card_cashback,NaN
...,...,...,...,...,...,...,...,...,...
192,2021-06-11 09:45:08,Card Rebate: Spotify,CRO,100.868755,EUR,9.99,10.785760,reimbursement,NaN
193,2021-06-03 23:25:12,Card Cashback,CRO,1.797610,EUR,0.20,0.215931,referral_card_cashback,NaN
194,2021-05-13 06:20:10,Card Cashback Reversal,CRO,-0.119895,EUR,0.02,0.021593,card_cashback_reverted,NaN
195,2021-05-11 09:45:21,Card Cashback,CRO,0.119895,EUR,0.02,0.021593,referral_card_cashback,NaN


In [59]:
sum(Cashbacks['Native Amount (EUR)'])

KeyError: 'Native Amount (EUR)'

In [ ]:
Cashbacks.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Crypto Cashback.csv')

In [ ]:
data = data[np.logical_not(data['Transaction Description'].isin(['From +358504077653', 'Supercharger Deposit (via app)']))].reset_index(drop=True)

data1 = data[['Timestamp (UTC)', 'Transaction Description', 'Currency', 'Amount', 'Native Amount (EUR)']].reset_index(drop=True)
data1.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']

data2 = data[['Timestamp (UTC)', 'Transaction Description', 'To Currency', 'To Amount', 'Native Amount (EUR)']]
data2.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']
data2 = data2[np.logical_not(data2.Change.isna())].reset_index(drop=True)

data = pd.concat([data1, data2], axis=0).reset_index(drop=True)

data["UTC_Time"] = pd.to_datetime(data["UTC_Time"], format='%m/%d/%Y %H:%M')
data = data.sort_values(by="UTC_Time")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')


data

In [ ]:
uniqueCoins = np.unique(data.Coin)
uniqueCoins

In [ ]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': data['Change'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'date_buy': data['UTC_Time'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'price_buy': data['Total Price (EUR)'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'amount_sell': data['Change'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'date_sell': data['UTC_Time'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'price_sell': (-data['Total Price (EUR)'][(data.Coin==coin) & (data.Change<0)]).to_list()}

In [ ]:
ALL

In [ ]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [ ]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [ ]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [ ]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Crypto Report.csv')

In [ ]:
ALL

In [ ]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

In [ ]:
260.80-219.15+92.64

In [ ]:
3022.4-1194.71+15.93 - 75.48 


In [53]:
sum(profit) + sum(Cashback['Total Price (EUR)']) -sum(loss)-sum(Fee['Total Price (EUR)']) - sum(Deposits['Fee (EUR)'])

1771.820000000013

In [54]:
sum(Withdraw['Total Price (EUR)'])-sum(Deposits['Total Price (EUR)'])

1607.670000000001

In [36]:
sum(Deposits['Total Price (EUR)'])

10418.68

In [37]:
sum(loss)

1198.8099999999986

In [38]:
sum(Fee['Total Price (EUR)']) + sum(Deposits['Fee (EUR)'])

80.84

In [348]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

ADA :    0.00000
ALICE :    0.00000
BAR :    0.00000
BNB :    0.00285
BTC :    0.00000
BUSD :    0.15344
DOGE :    0.00000
DOT :    0.00000
ENJ :    0.00000
ETH :    0.00000
EUR :    0.00031
FXS :    0.00000
HNT :    0.00000
IOTA :    0.00000
LTC :    0.00000
LUNA :    0.00000
MANA :    0.00000
MATIC :    0.00000
MTL :    0.00000
NFT :    18334.49575
RVN :    0.00000
SFP :    0.00000
SHIB :    0.00000
SOL :    0.00000
STMX :    0.00000
TRX :    0.00000
USDT :    0.00000
WABI :    0.14720
XLM :    0.00000
XRP :    0.00000
ZRX :    0.00000


# Etoro

In [161]:
data = pd.read_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro.csv')

In [162]:
data

,Position ID,Action,Amount,Units,Open Date,Close Date,Leverage,Spread,Profit,Open Rate,Close Rate,Take profit rate,Stop lose rate,Rollover Fees and Dividends,Copied From,Type,ISIN,Notes
0,798900619,Buy Boeing,100.00,0.465875,18/11/2020 15:33:00,01/12/2020 14:49:07,1,0.0,0.22,214.63,215.10,2361.13,0.01,0.0,-,Stocks,US0970231058,NaN
1,798046203,Buy Boeing,100.00,0.478515,17/11/2020 20:20:54,01/12/2020 14:39:45,1,0.0,2.95,208.98,215.14,2298.78,0.01,0.0,-,Stocks,US0970231058,NaN
2,673064189,Buy Amazon.com Inc,333.74,0.099928,13/07/2020 15:05:11,01/12/2020 14:36:05,1,0.0,-17.11,3339.84,3168.62,0.00,0.01,0.0,-,Stocks,US0231351067,NaN
3,799341020,Buy Boeing,50.00,0.243605,18/11/2020 20:32:06,01/12/2020 14:35:17,1,0.0,1.93,205.25,213.19,2257.75,0.01,0.0,-,Stocks,US0970231058,NaN
4,800480812,Buy Boeing,50.00,0.246427,19/11/2020 17:16:56,01/12/2020 14:35:09,1,0.0,2.54,202.90,213.19,2231.90,0.01,0.0,-,Stocks,US0970231058,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,804049275,"Buy Tesla Motors, Inc.",58.28,0.111948,23/11/2020 15:24:57,11/01/2021 20:18:50,1,0.0,32.74,520.60,813.06,5725.50,0.01,0.0,-,Stocks,US88160R1014,NaN
415,804055078,"Buy Tesla Motors, Inc.",50.00,0.096028,23/11/2020 15:26:45,11/01/2021 20:18:50,1,0.0,28.08,520.68,813.06,5727.49,0.01,0.0,-,Stocks,US88160R1014,NaN
416,798030691,"Buy Tesla Motors, Inc.",100.00,0.227433,17/11/2020 20:10:38,11/01/2021 20:18:50,1,0.0,84.92,439.69,813.06,4836.59,0.01,0.0,-,Stocks,US88160R1014,NaN
417,797571688,"Buy Tesla Motors, Inc.",100.00,0.224800,17/11/2020 16:05:11,11/01/2021 20:18:50,1,0.0,82.78,444.84,813.06,4893.24,0.01,0.0,-,Stocks,US88160R1014,NaN


In [158]:
USDEUR = []
for i, time in enumerate(data['Open Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [164]:
data['Amount (EUR)'] = np.round(USDEUR * data['Amount'],2)

In [168]:
USDEUR = []
for i, time in enumerate(data['Close Date']):
    USDEUR.append(get_rate("USD", "EUR",datetime.strptime(time, '%d/%m/%Y %H:%M:%S')))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [173]:
data['Amount Sell (EUR)'] = np.round((data['Amount'] + data['Profit']) * USDEUR,2)


data['Profit (EUR)'] = data['Amount Sell (EUR)'] - data['Amount (EUR)']

data['Action'] = [stock.replace('Buy ','') for stock in data['Action']]

In [180]:
data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro EUR.csv')

In [198]:
data_buy = data[['Action', 'Units', 'Open Date', 'Type', 'Amount (EUR)', 'Amount']]
data_buy.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)', 'Total Price (USD)']
data_buy['Action'] = ['Buy']*len(data_buy.Instrument)


data_sell = data[['Action', 'Units', 'Close Date', 'Type', 'Amount Sell (EUR)']]
data_sell.columns = ['Instrument', 'Units', 'Date', 'Type', 'Total Price (EUR)']
data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
data_sell['Action'] = ['Sell']*len(data_sell.Instrument)

data = pd.concat([data_buy, data_sell], axis=0)

data = data[['Type', 'Action', 'Instrument', 'Units', 'Date', 'Total Price (EUR)', 'Total Price (USD)']]

C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_buy['Action'] = ['Buy']*len(data_buy.Instrument)
C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sell['Total Price (USD)'] = data['Amount'] + data['Profit']
C:\Users\mmoghad\AppData\Local\Temp/ipykernel_13740/18286657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [200]:
data["Date"] = pd.to_datetime(data["Date"], format='%d/%m/%Y %H:%M:%S')
data = data.sort_values(by="Date")

data.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Buy Sell EUR.csv')

In [201]:
uniqueInstruments = np.unique(data.Instrument)
uniqueInstruments

array(['AIRBUS SE', 'Abbott Laboratories', 'Advanced Micro Devices Inc',
       'Air France-KLM', 'Alibaba', 'Alphabet', 'Amazon.com Inc',
       'American Airlines Group Inc', 'Apple', 'Aurora Cannabis Inc',
       'Ballard Power Systems Inc.', 'BioNano Genomics Inc', 'Bitcoin',
       'Boeing', 'Booking Holdings Inc', 'Boxlight Corp',
       'Build and Build', 'Cardano', 'Cocrystal Pharma Inc',
       'Delta Air Lines Inc (DE)', 'Dogecoin', 'Ethereum',
       'FuelCell Energy Inc', 'Globalstar', 'IOTA', 'Litecoin',
       'Meta Platforms Inc', 'Microsoft', 'Moderna Inc',
       'NVIDIA Corporation', 'Nio Inc.', 'Palantir Technologies Inc.',
       'Pfizer', 'Plug Power Inc', 'Tesla Motors, Inc.', 'Walmart Inc.',
       'Zoom Video Communications Inc', 'easyJet'], dtype=object)

In [228]:
ALL ={}

for instrument in uniqueInstruments:
    ALL[instrument] ={'amount_buy': data['Units'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'date_buy': data['Date'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'price_buy': data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Buy')].to_list(),
                      'amount_sell': (-data['Units'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list(),
                      'date_sell': data['Date'][(data.Instrument==instrument) & (data.Action == 'Sell')].to_list(),
                      'price_sell': (-data['Total Price (EUR)'][(data.Instrument==instrument) & (data.Action == 'Sell')]).to_list()}

In [229]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
stock = []
profit = []
loss = []
result = []
for instrument in uniqueInstruments:
    while (len(All_copy[instrument]['amount_buy']) != 0) and (len(All_copy[instrument]['amount_sell']) != 0):
        buy_date.append(All_copy[instrument]['date_buy'][0])
        sell_date.append(All_copy[instrument]['date_sell'][0])
        stock.append(instrument)
        if All_copy[instrument]['amount_buy'][0] <= abs(All_copy[instrument]['amount_sell'][0]):
            
            amount.append(All_copy[instrument]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[instrument]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[instrument]['price_sell'][0] * 
                                All_copy[instrument]['amount_buy'][0] / All_copy[instrument]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[instrument]['amount_sell'][0] += All_copy[instrument]['amount_buy'][0]
            All_copy[instrument]['price_sell'][0] += sellPricePortion
            
            if All_copy[instrument]['amount_sell'][0] == 0:
                del All_copy[instrument]['amount_sell'][0]
                del All_copy[instrument]['price_sell'][0]
                del All_copy[instrument]['date_sell'][0]
            del All_copy[instrument]['amount_buy'][0]
            del All_copy[instrument]['price_buy'][0]
            del All_copy[instrument]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[instrument]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[instrument]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[instrument]['price_buy'][0] * 
                                All_copy[instrument]['amount_sell'][0] / All_copy[instrument]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[instrument]['amount_buy'][0] += All_copy[instrument]['amount_sell'][0]
            All_copy[instrument]['price_buy'][0] -= buyPricePortion
            
            if All_copy[instrument]['amount_buy'][0] == 0:
                del All_copy[instrument]['amount_buy'][0]
                del All_copy[instrument]['price_buy'][0]
                del All_copy[instrument]['date_buy'][0]
            del All_copy[instrument]['amount_sell'][0]
            del All_copy[instrument]['price_sell'][0]
            del All_copy[instrument]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = abs(result *(result>=0))
loss = abs(result * (result<0))

In [230]:
Report =pd.DataFrame({'Stock': stock,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [231]:
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [260]:
Report.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report.csv')

In [245]:
Stocks = np.unique(data.Instrument[data.Type=='Stocks'])
Cryptos = np.unique(data.Instrument[data.Type=='Crypto'])

In [256]:
Report2020 = Report[Report['Sell Date'] <= datetime(2020,12,31,23,59,59)].reset_index(drop=True)
Report2021 = Report[Report['Sell Date'] > datetime(2020,12,31,23,59,59)].reset_index(drop=True)

In [257]:
Report_Stock = Report2021[Report2021.Stock.isin(Stocks)].reset_index(drop=True)

In [258]:
Report_Crypto = Report2021[Report2021.Stock.isin(Cryptos)].reset_index(drop=True)


In [259]:
Report2020.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Report 2020.csv')

Report_Stock.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Stock Report 2021.csv')
Report_Crypto.to_csv(r'C:\Users\mmoghad\Desktop\tax\Masoud Tax Files\Etoro Crypto Report 2021.csv')


In [65]:
get_rate("USD", "EUR",datetime(2021,4,20))

0.8298066550493735

In [66]:
0.829806655049373*25

20.745166376234327

In [368]:
datetime(2021,5,16).day

16

In [367]:
2225.57 + 2.76 - 1104.95 - 27.92 

1095.4600000000003

# Crypto.com

In [117]:
data = pd.read_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\crypto_transactions_record_20220529_232110.csv')

In [72]:
Cashbacks = data[data['Transaction Description'].isin(['CRO Stake Rewards', 'Card Cashback', 'Card Cashback Reversal', 'Card Rebate Reversal: Netflix', 
'Card Rebate: Netflix', 'Card Rebate: Spotify', 'Referral Bonus Reward', 'Sign-up Bonus Unlocked', 'Supercharger Reward'])].reset_index(drop=True)

Cashbacks = Cashbacks.drop(['To Currency', 'To Amount', 'Native Currency', 'Native Amount (in USD)', 'Transaction Kind', 'Transaction Hash'],axis=1)
Cashbacks



,Timestamp (UTC),Transaction Description,Currency,Amount,Native Amount (EUR)
0,30-12-2021 18:52,Card Cashback,CRO,2.278596,1.14
1,30-12-2021 17:32,Card Cashback,CRO,0.424724,0.21
2,30-12-2021 09:29,Card Cashback,CRO,0.557017,0.28
3,30-12-2021 08:17,Card Cashback,CRO,0.338780,0.17
4,29-12-2021 19:07,Card Cashback,CRO,0.642344,0.32
...,...,...,...,...,...
192,11-06-2021 09:45,Card Rebate: Spotify,CRO,100.868755,9.99
193,03-06-2021 23:25,Card Cashback,CRO,1.797610,0.20
194,13-05-2021 06:20,Card Cashback Reversal,CRO,-0.119895,0.02
195,11-05-2021 09:45,Card Cashback,CRO,0.119895,0.02


In [73]:
sum(Cashbacks['Native Amount (EUR)'])

227.31657899000012

In [74]:
Cashbacks.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Crypto Cashback.csv')

In [79]:
data

,Timestamp (UTC),Transaction Description,Currency,Amount,To Currency,To Amount,Native Currency,Native Amount (EUR),Native Amount (in USD),Transaction Kind,Transaction Hash
0,30-12-2021 18:52,Card Cashback,CRO,2.278596,NaN,NaN,EUR,1.14,1.230807,referral_card_cashback,NaN
1,30-12-2021 17:32,Card Cashback,CRO,0.424724,NaN,NaN,EUR,0.21,0.226728,referral_card_cashback,NaN
2,30-12-2021 09:29,Card Cashback,CRO,0.557017,NaN,NaN,EUR,0.28,0.302304,referral_card_cashback,NaN
3,30-12-2021 08:17,Card Cashback,CRO,0.338780,NaN,NaN,EUR,0.17,0.183541,referral_card_cashback,NaN
4,29-12-2021 19:07,Card Cashback,CRO,0.642344,NaN,NaN,EUR,0.32,0.345490,referral_card_cashback,NaN
...,...,...,...,...,...,...,...,...,...,...,...
209,11-05-2021 09:45,Card Cashback,CRO,0.119895,NaN,NaN,EUR,0.02,0.021593,referral_card_cashback,NaN
210,11-05-2021 09:07,Buy SHIB,SHIB,1000000.000000,NaN,NaN,EUR,28.55,30.824168,crypto_purchase,NaN
211,20-04-2021 18:54,Sign-up Bonus Unlocked,CRO,351.146656,NaN,NaN,USD,20.75,25.000000,referral_gift,NaN
212,20-04-2021 18:53,CRO Stake,CRO,-2149.000000,NaN,NaN,EUR,350.11,377.998231,lockup_lock,NaN


In [118]:
# data = data[np.logical_not(data['Transaction Description'].isin(['From +358504077653', 'Supercharger Deposit (via app)']))].reset_index(drop=True)

data1 = data[['Timestamp (UTC)', 'Transaction Description', 'Currency', 'Amount', 'Native Amount (EUR)']].reset_index(drop=True)
data1.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']

data2 = data[['Timestamp (UTC)', 'Transaction Description', 'To Currency', 'To Amount', 'Native Amount (EUR)']]
data2.columns = ['UTC_Time', 'Transaction Description', 'Coin', 'Change', 'Total Price (EUR)']
data2 = data2[np.logical_not(data2.Change.isna())].reset_index(drop=True)

data = pd.concat([data1, data2], axis=0).reset_index(drop=True)
data = data[np.logical_not(data['Transaction Description'].isin(['CRO Stake', 'Supercharger Deposit (via app)', 
                                                                 'Supercharger Withdrawal (via app)', 'Card Cashback Reversal',
                                                                 'Supercharger Stake (via app)']))].reset_index(drop=True)

data["UTC_Time"] = pd.to_datetime(data["UTC_Time"], format='%d-%m-%Y %H:%M').reset_index(drop=True)
data = data.sort_values(by="UTC_Time")

data.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Etoro Buy Sell EUR.csv')


data

,UTC_Time,Transaction Description,Coin,Change,Total Price (EUR)
204,2021-04-20 18:44:00,EUR -> CRO,CRO,2150.170000,350.00
202,2021-04-20 18:44:00,EUR -> CRO,EUR,-350.000000,350.00
201,2021-04-20 18:54:00,Sign-up Bonus Unlocked,CRO,351.146656,20.75
200,2021-05-11 09:07:00,Buy SHIB,SHIB,1000000.000000,28.55
199,2021-05-11 09:45:00,Card Cashback,CRO,0.119895,0.02
...,...,...,...,...,...
4,2021-12-29 19:07:00,Card Cashback,CRO,0.642344,0.32
3,2021-12-30 08:17:00,Card Cashback,CRO,0.338780,0.17
2,2021-12-30 09:29:00,Card Cashback,CRO,0.557017,0.28
1,2021-12-30 17:32:00,Card Cashback,CRO,0.424724,0.21


In [119]:
uniqueCoins = np.unique(data.Coin)
uniqueCoins

array(['AXS', 'CRO', 'DYDX', 'EUR', 'HBAR', 'SDN', 'SHIB'], dtype=object)

In [120]:
ALL ={}

for coin in uniqueCoins:
    ALL[coin] ={'amount_buy': data['Change'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'date_buy': data['UTC_Time'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'price_buy': data['Total Price (EUR)'][(data.Coin==coin) & (data.Change>0)].to_list(),
                'amount_sell': data['Change'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'date_sell': data['UTC_Time'][(data.Coin==coin) & (data.Change<0)].to_list(),
                'price_sell': (-data['Total Price (EUR)'][(data.Coin==coin) & (data.Change<0)]).to_list()}

In [121]:
ALL

{'AXS': {'amount_buy': [0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415],
  'date_buy': [Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-07 03:19:00'),
   Timestamp('2021-10-07 23:21:00'),
   Timestamp('2021-10-09 01:40:00'),
   Timestamp('2021-10-10 01:18:00'),
   Timestamp('2021-10-11 01:14:00'),
   Timestamp('2021-10-12 01:02:00'),
   Timestamp('2021-10-13 04:18:00'),
   Timestamp('2021-10-14 01:43:00'),
   Timestamp('2021-10-15 01:02:00'),
   Timestamp('2021-10-16 01:02:00'),
   Timestamp('2021-10-17 01:02:00')

In [122]:
All_copy = ALL.copy()

buy_date = []
amount = []
sell_date = []
buy_price = []
sell_price = []
currency = []
profit = []
loss = []
result = []
for coin in uniqueCoins:
    while (len(All_copy[coin]['amount_buy']) != 0) and (len(All_copy[coin]['amount_sell']) != 0):
        buy_date.append(All_copy[coin]['date_buy'][0])
        sell_date.append(All_copy[coin]['date_sell'][0])
        currency.append(coin)
        if All_copy[coin]['amount_buy'][0] <= abs(All_copy[coin]['amount_sell'][0]):
            
            amount.append(All_copy[coin]['amount_buy'][0])
        
            buy_price.append(np.round(All_copy[coin]['price_buy'][0],2))
            
            sellPricePortion = (All_copy[coin]['price_sell'][0] * 
                                All_copy[coin]['amount_buy'][0] / All_copy[coin]['amount_sell'][0])
            
            sell_price.append(np.round(sellPricePortion,2))
            
            All_copy[coin]['amount_sell'][0] += All_copy[coin]['amount_buy'][0]
            All_copy[coin]['price_sell'][0] += sellPricePortion
            
            if All_copy[coin]['amount_sell'][0] == 0:
                del All_copy[coin]['amount_sell'][0]
                del All_copy[coin]['price_sell'][0]
                del All_copy[coin]['date_sell'][0]
            del All_copy[coin]['amount_buy'][0]
            del All_copy[coin]['price_buy'][0]
            del All_copy[coin]['date_buy'][0]
            
        else:
            
            amount.append(-All_copy[coin]['amount_sell'][0])
        
            sell_price.append(np.round(-All_copy[coin]['price_sell'][0],2))
            
            buyPricePortion = (-All_copy[coin]['price_buy'][0] * 
                                All_copy[coin]['amount_sell'][0] / All_copy[coin]['amount_buy'][0])
            
            buy_price.append(np.round(buyPricePortion,2))
            
            All_copy[coin]['amount_buy'][0] += All_copy[coin]['amount_sell'][0]
            All_copy[coin]['price_buy'][0] -= buyPricePortion
            
            if All_copy[coin]['amount_buy'][0] == 0:
                del All_copy[coin]['amount_buy'][0]
                del All_copy[coin]['price_buy'][0]
                del All_copy[coin]['date_buy'][0]
            del All_copy[coin]['amount_sell'][0]
            del All_copy[coin]['price_sell'][0]
            del All_copy[coin]['date_sell'][0]

            
result = np.subtract(sell_price, buy_price)
profit = result *(result>=0)
loss = -result * (result<0)

In [123]:
Report =pd.DataFrame({'Coin': currency,
                      'Buy Date': buy_date,
                      'Amount':  amount,
                      'Buy Price': buy_price,
                      'Sell Date': sell_date,
                      'Sell Price': sell_price,
                      'Loss': loss,
                      'Profit': profit})

In [124]:
Report["Buy Date"] = pd.to_datetime(Report["Buy Date"], format='%m/%d/%Y %H:%M')
Report["Sell Date"] = pd.to_datetime(Report["Sell Date"], format='%m/%d/%Y %H:%M')
Report = Report.sort_values(by="Buy Date")
Report = Report.sort_values(by="Sell Date").reset_index(drop=True)
Report = Report[np.logical_and(Report['Buy Price'] != 0, Report['Sell Price'] != 0)].reset_index(drop=True)

In [126]:
Report.to_csv(r'C:\Users\JKHATRI\Downloads\Tax Documents\Cryptocom\Crypto Report.csv')

In [127]:
ALL

{'AXS': {'amount_buy': [0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415,
   0.00154415],
  'date_buy': [Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-06 16:39:00'),
   Timestamp('2021-10-07 03:19:00'),
   Timestamp('2021-10-07 23:21:00'),
   Timestamp('2021-10-09 01:40:00'),
   Timestamp('2021-10-10 01:18:00'),
   Timestamp('2021-10-11 01:14:00'),
   Timestamp('2021-10-12 01:02:00'),
   Timestamp('2021-10-13 04:18:00'),
   Timestamp('2021-10-14 01:43:00'),
   Timestamp('2021-10-15 01:02:00'),
   Timestamp('2021-10-16 01:02:00'),
   Timestamp('2021-10-17 01:02:00')

In [128]:
for coin in ALL:
    print(coin , ':   ', "{0:.5f}".format(sum(ALL[coin]['amount_buy'])))

AXS :    0.04632
CRO :    5788.37114
DYDX :    0.22285
EUR :    0.00000
HBAR :    3.91069
SDN :    0.29616
SHIB :    2000000.00000


In [129]:
260.80-219.15+92.64

134.29000000000002

In [56]:
3022.4-1194.71+15.93 - 75.48 


1768.14